In [1]:
%load_ext autoreload
%autoreload 2

In [55]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from htools import *
from incendio.layers import mish

In [176]:
bs = 2
n_seq = 4
n_emb = 6

In [211]:
x = torch.randn(bs, n_seq, n_emb)
x.shape

torch.Size([2, 4, 6])

In [212]:
lin = nn.Linear(n_emb, n_emb*3)
qkv = lin(x)
qkv.shape

torch.Size([2, 4, 18])

In [214]:
q, k, v = qkv.chunk(3, dim=-1)

In [217]:
q.shape

torch.Size([2, 4, 6])

In [218]:
qk = q @ k.transpose(-2, -1)
qk.shape

torch.Size([2, 4, 4])

In [219]:
w = F.softmax(qk/np.sqrt(k.shape[-1]), dim=-1)
w.shape

torch.Size([2, 4, 4])

In [220]:
w.shape, v.shape

(torch.Size([2, 4, 4]), torch.Size([2, 4, 6]))

In [221]:
w

tensor([[[0.3222, 0.2185, 0.2588, 0.2005],
         [0.4319, 0.1282, 0.1905, 0.2495],
         [0.3373, 0.1951, 0.2251, 0.2425],
         [0.3924, 0.1480, 0.2008, 0.2588]],

        [[0.2347, 0.2317, 0.2986, 0.2350],
         [0.2348, 0.2368, 0.3028, 0.2256],
         [0.2268, 0.3190, 0.2268, 0.2274],
         [0.2319, 0.2344, 0.3131, 0.2206]]], grad_fn=<SoftmaxBackward>)

In [222]:
v

tensor([[[-0.1854, -0.3212,  0.7594,  0.3210, -0.1596,  0.8336],
         [ 0.1120, -0.0854, -0.7435,  0.2728,  0.0849, -0.6934],
         [-1.2247,  0.5296, -0.5358, -0.6297,  0.9698, -0.6206],
         [-0.9533, -0.5769, -0.1699, -0.1935,  0.5883,  0.3895]],

        [[-1.4310,  0.2038, -0.4679, -0.6899,  1.0920, -0.3128],
         [-1.1908,  0.5564, -0.2625, -0.5566,  0.8116, -0.3657],
         [-0.3030,  0.0169, -0.1577,  0.3909,  0.1791, -0.3857],
         [-1.0234, -0.2025, -0.4626, -0.6421,  1.0326, -0.1737]]],
       grad_fn=<SplitBackward>)

In [223]:
res = w @ v
res.shape

torch.Size([2, 4, 6])

In [235]:
class Projector(nn.Module):
    
    def __init__(self, n_in, n_out_single, spaces=3):
        super().__init__()
        self.spaces = spaces
        self.n_out_single = n_out_single
        self.fc = nn.Linear(n_in, n_out_single*spaces)
        
    def forward(self, x):
        return self.fc(x).chunk(self.spaces, dim=-1)

In [421]:
class DotProductAttention(nn.Module):
    
    def __init__(self, n_in, n_f_single, n_out, n_heads, scale=True, 
                 drop_p1=0.1, drop_p2=0.1, return_attn=False):
        super().__init__()
        assert n_f_single % n_heads == 0, \
            'n_f_single must be divisible by n_heads'
        self.proj_in = Projector(n_in, n_f_single, spaces=3)
        self.drop_in = nn.Dropout(drop_p1)
        self.proj_out = nn.Linear(n_f_single, n_out)
        self.drop_out = nn.Dropout(drop_p2)
        
        # Non-layer attributes.
        self.n_heads = n_heads
        self.scale = scale
        self.drop_p1 = drop_p1
        self.drop_p2 = drop_p2
        self.return_attn = return_attn
        
    def forward(self, x):
        q, k, v = map(self.split_heads, self.proj_in(x))
        scores = q @ k.transpose(-2, -1)
        if self.scale: scores /= np.sqrt(scores.shape[-1])
        weights = F.softmax(scores, dim=-1)
        # Note: Huggingface returns weights post dropout which seems a little
        # odd. Maybe For now I'm just experimenting so I'm skipping that part.
        x = self.drop_in(weights) @ v
        x = self.merge_heads(x)
        x = self.drop_out(self.proj_out(x))
        return (x, weights) if self.return_attn else x
    
    def split_heads(self, inp):
        # Reshape input so hidden_dim is split equally between n heads.
        bs, seq, hid = inp.shape
        return inp.reshape(bs, self.n_heads, seq, hid // self.n_heads)
    
    def merge_heads(self, inp):
        bs, heads, seq, hid = inp.shape
        return inp.permute(0, 2, 1, 3).reshape(bs, seq, -1)
    
# class MultiHeadAttention(DotProductAttention):
    
#     def __init__(self, n_in, n_f_single, n_heads, scale=True, drop_p=0.1,
#                  return_attn=False):
#         super().__init__(n_in, n_f_single, scale, drop_p, return_attn)
#         self.n_heads = n_heads
        
#     def forward(self, x):
#             q, k, v = map(self.split_heads, x)
#             return super().forward((q, k, v))

In [417]:
proj = Projector(n_emb, n_emb)
qkv = proj(x)
attrmap('shape', *qkv)

[torch.Size([2, 4, 6]), torch.Size([2, 4, 6]), torch.Size([2, 4, 6])]

In [418]:
print(x.shape)
attn = DotProductAttention(n_emb, n_emb, n_emb*2, 1)
res = attn(x)
print(res.shape)
res

torch.Size([2, 4, 6])
torch.Size([2, 4, 12])


tensor([[[-0.0219,  0.1480, -0.0779, -0.3398,  0.2438,  0.0000,  0.3983,
          -0.3061,  0.0000,  0.0000,  0.2716, -0.2372],
         [-0.0094,  0.1142,  0.0164, -0.3359, -0.0000,  0.1062,  0.4832,
          -0.4665,  0.7347,  0.0000,  0.0475,  0.0112],
         [ 0.0257,  0.0845, -0.0568, -0.3908,  0.2509,  0.1602,  0.3955,
          -0.0000,  0.5900,  0.0692,  0.3223, -0.2070],
         [-0.0941,  0.1840, -0.0285, -0.3601, -0.0759,  0.0678,  0.4196,
          -0.4413,  0.0000,  0.0787, -0.0043,  0.0018]],

        [[ 0.0991, -0.1446,  0.0258, -0.5993,  0.0666,  0.2172,  0.3595,
          -0.4314,  1.0157,  0.1354,  0.3244,  0.0982],
         [ 0.0345, -0.0287,  0.0219, -0.0000, -0.0130,  0.1340,  0.3629,
          -0.4824,  0.9026,  0.1333,  0.2206,  0.0440],
         [ 0.1349, -0.1238, -0.0000, -0.4414,  0.2259,  0.3804,  0.5059,
          -0.1504,  0.8223,  0.1075,  0.0000,  0.0233],
         [ 0.0000, -0.0000, -0.0891, -0.4575,  0.2274,  0.3732,  0.4891,
          -0.0000,  0.

In [419]:
attn = DotProductAttention(n_emb, n_emb*2, n_emb, n_heads=3)
res = attn(x)
res.shape

torch.Size([2, 4, 6])

In [145]:
class FanForward(nn.Module):
    """Fan out and back in. Not really sure why though since we end up with 
    the same shape."""
    
    def __init__(self, n_in, hidden_scalar, drop_p=0.1, act=mish):
        super().__init__()
        n_hid = n_in * hidden_scalar
        self.fc1 = nn.Linear(n_in, n_hid)
        self.fc2 = nn.Linear(n_hid, n_in)
        self.act = act
        self.drop = nn.Dropout(drop_p)
        
    def forward(self, x):
        x = self.act(self.fc1(x))
        return self.drop(self.act(self.fc2(x)))

In [423]:
class TransformerDecoder(nn.Module):
    
    def __init__(self):
        super().__init__()
#         DotProductAttention()
#         FanForward()
        
    def forward(self, x):
        return x

In [84]:
FanForward(5, 6)

FanForward(
  (fc1): Linear(in_features=5, out_features=30, bias=True)
  (fc2): Linear(in_features=30, out_features=5, bias=True)
  (drop): Dropout(p=0.1, inplace=False)
)

In [81]:
ff = FanForward(n_emb, 4)
ff(x).shape

torch.Size([2, 4, 12])


torch.Size([2, 4, 3])

In [156]:
from transformers.modeling_gpt2 import Attention

In [166]:
config = Args(n_head=1, attn_pdrop=.1, resid_pdrop=.1, n_ctx=n_emb*2, 
              n_layer=1, n_embd=n_emb, layer_norm_epsilon=1e-3,
              output_attentions=False)
attn_gpt = Attention(n_emb, n_emb*2, config)
res_gpt = attn_gpt(x)
len(res_gpt)

2

In [169]:
res_gpt[-2].shape

torch.Size([2, 4, 3])

In [171]:
attn_gpt.c_attn(x).shape

torch.Size([2, 4, 9])

In [61]:
# FROM BLOG POST
class Conv1D(nn.Module):
    def __init__(self, nx, nf):
        super().__init__()
        self.nf = nf
        w = torch.empty(nx, nf)
        nn.init.normal_(w, std=0.02)
        self.weight = nn.Parameter(w)
        self.bias = nn.Parameter(torch.zeros(nf))

    def forward(self, x):
        size_out = x.size()[:-1] + (self.nf,)
        x = torch.addmm(self.bias, x.view(-1, x.size(-1)), self.weight)
        x = x.view(*size_out)
        return x
    
class FeedForward(nn.Module):
    def __init__(self, dropout, d_model=768, nx=768*4):
        super().__init__()
        self.c_fc    = Conv1D(d_model, nx)
        self.c_proj  = Conv1D(nx, d_model)
        self.act     = F.gelu
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        x = self.act(self.c_fc(x))
        print(x.shape)
        return self.dropout(self.c_proj(x))

In [62]:
FeedForward(0, n_emb, n_emb*4)(x).shape

torch.Size([2, 4, 12])


torch.Size([2, 4, 3])

In [54]:
w.shape, v.shape

(torch.Size([2, 4, 4]), torch.Size([2, 4, 3]))

In [27]:
z = torch.randn(bs, n_seq, n_emb, n_emb)
z.shape

torch.Size([2, 4, 3, 3])

In [30]:
lin2 = nn.Linear(3, 5)
lin2(z).shape

torch.Size([2, 4, 3, 5])

In [33]:
lin2.weight

Parameter containing:
tensor([[-0.5565, -0.2874, -0.5044],
        [ 0.3084,  0.2368, -0.2929],
        [-0.1206,  0.0814, -0.2280],
        [ 0.1191,  0.0834, -0.2181],
        [-0.0737,  0.3648,  0.3930]], requires_grad=True)

In [34]:
z.dim()

4